# Introduction to Python - Numpy/Scipy

## Problem Set

In [1]:
# Author: Alex Schmitt (schmitt@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2017-03-15 15:25:59.147591


### Exercise 1

Consider the polynomial expression
\begin{equation*}
	p(x) = a_0 + a_1 x + a_2 x^2 + \cdots + a_N x^N = \sum_{n=0}^N a_n x^n,
\end{equation*}
where $x$ is a scalar number.

1. Write a Python function **poly(x, coeff)** that implements $p(x)$ (i.e. evaluates the polynomial) given a point **x** and a list of coefficients **coeff** ($= a_0,...,a_N$). Use a loop with **enumerate()**.

2. Write a new function **poly_np(x, coeff)** that does the same job, but uses NumPy arrays and vectorized operations, rather than any form of Python loop. Hint: Use **np.cumprod()**.

3. Using **%time**, check how long it takes for both functions to run when **len(coeff)** = 50 (you can use arbitrary values for the elements in **coeff**).


### Answers

#### (a)

As an example, let's compute $p(x) = 1 + x^2 + x^4$ at $x = 2$, which gives $p(2) = 21$. Using **enumerate()**, the loop is straightforward:

In [8]:
def poly(x, coeff):
    # initialize s as 0
    s = 0
    for (i, a) in enumerate(coeff):
        # for each iteration, add the corresponding term to s
        s = s + a * x**i
        
    return s

poly(2, [1, 2, 4])

21

Note that we can implement the exact same algorithm in one line:

In [40]:
# def poly(x, coeff):
#     return sum(a * x**i for i, a in enumerate(coeff))

# poly(2, [1, 2, 4])

#### (b) 

Using **np.cumprod()** on a sequence gives an array where at each position, you have the product of all elements that come before or at that position in the original array. For example, **np.cumprod([2,3,4,5])** would return **[2,6,24,120]** as a NP array. For the function here, we need to apply **np.cumprod()** on an array **[1, x, x, x, ..]**, which would return **[1, x, x^2, x^3,...]**. We can use the inner product of this array and **coeff**.

In [9]:
def poly_np(x, coeff):
    # define X = [1 x x ...]
    X = np.empty(len(coeff))
    X[0] = 1
    X[1:] = x
    # using cumprod on X gives [1 x x^2 x^3 ...]
    y = np.cumprod(X)
    # compute and return inner product
    return coeff @ y

poly_np(2, [1, 2, 4])

21.0

As an alternative, note that a vectorized operations also works if *the array is in the exponent*. In other words, implementing something like $x^{(0,1,2)}$ in Python, where $x$ is a scalar, would give $(x^0, x^1, x^2)$. Again, this works only if the sequence $(0,1,2)$ is a NumPy array. 

In [13]:
def poly_np2(x, coeff):
    return coeff @ x**np.linspace(0,len(coeff)-1,len(coeff))

poly_np2(2, [1, 2, 4])

21.0

#### (c) 

In [14]:
# use random module to assign arbitrary values to coeff
N = 50000
coeff = np.random.uniform(0, 1, N)
x = 0.1

With **len(coeff) = 50**, the running times across all functions are very small and close to each other. Instead, let's increase the number of elements in **coeff** to 50000. There are notable differences in the running times across the functions, with the **poly_np** function using **np.cumprod()** being by far the fastest - the difference compared to the loop-implementation in Vanilla Python is a factor of about 30 -- another example for the speed of vectorized operations.

In [15]:
%time print(poly(x, coeff))
%time print(poly_np(x, coeff))
%time print(poly_np2(x, coeff))

0.201990823858
CPU times: user 25 ms, sys: 321 µs, total: 25.4 ms
Wall time: 25.1 ms
0.201990823858
CPU times: user 861 µs, sys: 204 µs, total: 1.06 ms
Wall time: 576 µs
0.201990823858
CPU times: user 12.3 ms, sys: 471 µs, total: 12.7 ms
Wall time: 3.08 ms


### Exercise 2

In this question, you are asked to write your own implementation of the *bisection method* to compute the root of a real-valued function. Of course, most programming languages already have in-built implementations (e.g. in SciPy: **scipy.optimize.bisect**}, so writing your own function may seem a bit redundant. The goal here is to give you some idea about the inner workings of many of the algorithms used in scientific computing. The bisection method, while quite simple, captures two important features of most root-finding and optimization methods: it is a *local* method and it is based on an *iterative procedure*. 

- Recall that a root of a real-valued function $f$ is a value $x$ in the domain of the function such that $f(x)=0$. The idea behind the bisection method is based on the *Intermediate Value Theorem*: if $f$ is continuous and defined on the interval $[a,b]$, and if $f(a)$ and $f(b)$ are distinct values, then $f$ must assume all values in between. Since we are interested in where $f$ assumes the value 0, we need $f(a)$ and $f(b)$ to have different signs.
- The bisection method implements the following "pseudo-code":

(i) Start with two values $a$ and $b$ such that $f(a)$ and $f(b)$ are defined and have different signs. Moreover, specify a "tolerance level" $tol$ which should be a very small number, e.g. 1e-8.

(ii) Compute the midpoint between $a$ and $b$, $x = \frac{a + b}{2}$. 

(iii) If $f(x)$ has the same sign as $f(a)$, replace the left endpoint of the interval with $x$, i.e. $a = x$.

(iv) If $f(x)$ has the same sign as $f(b)$, replace the right endpoint of the interval with $x$, i.e. $b = x$.

(v) Repeat from (ii) until the absolute value of $f(x)$ is less than $tol$, i.e. $|f(x)| < tol$.

- Bisection is an *iterative procedure*: at the beginning of each iteration step, the interval $[a,b]$ contains a root of $f$. The interval is then divided ("bisected") into two subintervals of equal length. One of the two subintervals must contain the root, and hence have endpoints of different signs. This subinterval is taken as the interval $[a,b]$ used for the next iteration. This process continues until the resulting midpoint $x$ of the current interval is sufficiently close to 0.  
- Moreover, note that bisection is a *local* method: it will not give you all the roots of a function, but only one of the roots (in case there are multiple roots) between $a$ and $b$. A corollary of this is that the outcome of bisection (and of local methods in general) is sensitive to the starting point chosen by the user, here the values for $a$ and $b$.

Write a function **mybisect(f, a, b)** in Python that implements the pseudo-code above. Then, test it on the function 
\begin{equation*}
    f(x) = \sin(4 (x - 1/4)) + x + x^{20} - 1,
\end{equation*}
i.e. find a root of this function. Compare your result to what SciPy's in-built function returns. 

*Hint*: most modern programming languages have some type of **while**-loop, which will prove useful here. Moreover, in Python/NumPy, consider using the **abs()** and **np.sign()** functions.  


### Answers

In [2]:
# function to use bisection on
def fun(x):
    return np.sin(4 * (x - 0.25)) + x + x**20 - 1


def mybisect(fun, a, b):
    """
    Implements the bisection method
    """
    # choose tolerance level
    tol = 1e-10
    # initialize d 
    d = 1
    # while-loop: iterate until d sufficiently small
    while abs(d) > tol:
        # find intermediate value between a and b
        x = (a + b)/2
        # evaluate function
        d = fun(x)
        # find new end points for interval [a,b]
        if np.sign(d) == np.sign(fun(a)):
            a = x
        elif np.sign(d) == np.sign(fun(b)):
            b = x
    
    return x

print(mybisect(fun,0,2))        
print(scipy.optimize.bisect(fun,0,2))
        

NameError: name 'np' is not defined

### Exercise 3 -> ADD QUESTION FROM CLIMATE_ECON PS 4 

Solve the model in the previous question (without the externality) numerically for a more general setting, in particular with a CIES utility function and less than full depreciation. Choose reasonable values for the parameters. How does the optimal extraction profile depend on the growth rate of total factor productivity?

### Answer
Solving the model numerically is straightforward. The code should look familiar from the two-period model we have seen in previous lectures. In particular, we can again use Scipy's minimization routine (**scipy.optimize.minimize**).

Note that since there is no extraction or externality cost caused by fossil fuel use, the complete resource stock will be used over the two periods.

In [ ]:
## Define functions
def u(x, nu):
    """
    Utility function
    """
    if nu == 1: # logarithmic utility
        return np.log(x)
    else:
        return (1 - nu)**(-1) * x**(1-nu)
    
def F(k, e, alpha, A):
    """
    Production function: Cobb-Douglas
    """
    return A * k**(alpha) * e**(1-alpha)

def obj(x, alpha, A1, g, beta, delta, nu, K1):
    """
    Computes the social planner's objective function, i.e. the representative agent's lifetime utility
    """
    E1, I = x
    return - u(F(K1, E1, alpha, A1) - I, nu) \
            - beta * u( F( (1-delta) * K1 + I, R - E1, alpha, (1 + g) * A1) + (1 - delta) * ( (1-delta) * K1 + I), nu)


## Set parameters
alpha = 0.6
beta = 0.98
R = 1
K1 = 1
A1 = 1
nu = 2
delta = 0.9

## Closed-form solution (with log utility and full depreciation), used as initial guess below
E1 = R * (1 + alpha * beta)/(1 + alpha * beta + beta)
K2 = alpha * beta * F(K1, E1, alpha, A1) / (1 + alpha * beta)
x0 = [E1, K2]
print(x0)

## determine bounds for choice variables
bnds = [(1e-8, 1e+12), (1e-8, 1e+12)]


## Baseline
g = 0.1
# run minimization routine
res = scipy.optimize.minimize(obj, x0,\
                              args = tuple([alpha, A1, g, beta, delta, nu, K1]), \
                              bounds = bnds, method='SLSQP', tol = 1e-12)
print(res.x)

## Increase productivity growth
g = 0.2
res = scipy.optimize.minimize(obj, x0,\
                              args = tuple([alpha, A1, g, beta, delta, nu, K1]), \
                              bounds = bnds, method='SLSQP', tol = 1e-12)
print(res.x)

We can see that less than full depreciation and non-logarithmic utility, higher productivity growth leads to less investment and hence a lower capital stock in period 2. This is intuitive: recall that $\nu > 1$ indicates a lower elasticity of intertemporal substitution (or a higher aversion to intertemporal consumption inequality), hence if the household is richer in the future (due to a higher productivity), the incentive to save today is weaker. A lower capital stock in period 2 implies that the marginal return to capital - $F_K(K_2, E_2) + 1 - \delta$ - increases. By the Hotelling rule, this implies that the *relative* marginal return to fossil fuel in period 2 (relative to period 1) has to rise as well. This can be achieved by shifting extraction from period 2 to period 1, thus leading to a (albeit small) increase in current fossil fuel use.